# Encontrando insights de la UEFA Champions League

La Liga de Campeones de la UEFA, a menudo llamada Liga de Campeones, es una competencia anual de fútbol por excelencia que cautiva a los fanáticos de todo el mundo. Establecida en 1955 como la Copa de Clubes Campeones de Europa, evolucionó a la Liga de Campeones de la UEFA en 1992, ampliando su atractivo. El formato moderno cuenta con 32 equipos de clubes de primer nivel seleccionados en función de su desempeño en la liga nacional, lo que aumenta la intriga.  

Este evento electrizante trasciende los deportes y se convierte en una celebración de la unidad, la cultura y el orgullo nacional. Los fanáticos, vestidos con los colores de sus países, crean una atmósfera eléctrica, lo que hace que el torneo sea tanto sobre los espectadores como sobre los jugadores. Financieramente, la Liga de Campeones es un salvavidas para los clubes, ya que aumenta los ingresos y ofrece oportunidades transformadoras. Sin embargo, genera debates sobre las disparidades de riqueza en el fútbol europeo.  

La Liga de Campeones es sinónimo de rivalidades históricas, triunfos de los desvalidos y brillantez individual. Para los jugadores, representa la cima de su carrera, mientras que para los fanáticos, es un fenómeno cultural. El himno y los rituales icónicos enriquecen la experiencia futbolística. En 200 palabras, la UEFA Champions League es el epítome de la excelencia del fútbol europeo, que ofrece momentos inolvidables, recompensas económicas y un impacto cultural único, con 32 clubes de primer nivel que se suman a su atractivo.

**Nombres de las tablas o ficheros a utilizar**:  
[uefa_2020.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/EQuW8d16tZJKpYZN6jR480sBgc1IYypAw9hGHKQARS560g?e=7TybBU)  
[uefa_2021.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/ESG8YBrjD3tBky3aGT0MwIABNt_PQxFVVYVlWMnZYUmV8g?e=VrYHtX)  
[uefa_2022.csv](https://tajamar365.sharepoint.com/:x:/s/3405-MasterIA2024-2025/Ecf8O8U8sxJLoW9Bkd2ZVzgBb_gZc35mNcprnR9FMlzliQ?e=Vn3a6v)  
- Todas las tablas tienen las mismas columnas y data types

| Column | Definition | Data type |
|--------|------------|-----------|
| `STAGE`| Stage of the March | `VARCHAR(50)` |
| `DATE` | When the match occurred. | `DATE` |
| `PENS` | Did the match end with penalty | `VARCHAR(50)` |
| `PENS_HOME_SCORE` | In case of penalty, score by home team | `VARCHAR(50)` |
| `PENS_AWAY_SCORE` | In case of penalty, score by away team | `VARCHAR(50)` |
| `TEAM_NAME_HOME` | Team home name | `VARCHAR(50)` |
| `TEAM_NAME_AWAY`| Team away  name | `VARCHAR(50)` |
| `TEAM_HOME_SCORE` | Team home score | `NUMBER` |
| `TEAM_AWAY_SCORE` | Team away score | `NUMBER` |
| `POSSESSION_HOME` | Ball possession for the home team | `FLOAT` |
| `POSSESSION_AWAY` | Ball possession for the away team | `FLOAT` |
| `TOTAL_SHOTS_HOME` | Number of shots by the home team | `NUMBER` |
| `TOTAL_SHOTS_AWAY` | Number of shots by the away team | `NUMBER`
| `SHOTS_ON_TARGET_HOME` | Total shot for home team | `FLOAT` |
| `SHOTS_ON_TARGET_AWAY` | Total shot for away team | `FLOAT` |
| `DUELS_WON_HOME` | duel win possession of ball - for home team | `NUMBER` |
| `DUELS_WON_AWAY` | duel win possession of ball - for away team | `NUMBER` 
| `PREDICTION_TEAM_HOME_WIN` | Probability of home team to win | `FLOAT` |
| `PREDICTION_DRAW` | Probability of draw | `FLOAT` |
| `PREDICTION_TEAM_AWAY_WIN` | Probability of away team to win | `FLOAT` |
| `LOCATION` | Stadium where the match was held | `VARCHAR(50)` |

### Requerimientos:  
-Puedes utilizar SQL o PySpark

In [0]:
from pyspark.sql import (
    SparkSession,
    types,
    functions as F,
)

spark = (
    SparkSession
    .builder
    .appName('Caso_2')
    .getOrCreate()
)

df_uefa_2020 = spark.read.csv('/FileStore/Caso_2/uefa_2020.csv', header=True, inferSchema=True)
df_uefa_2021 = spark.read.csv('/FileStore/Caso_2/uefa_2021.csv', header=True, inferSchema=True)
df_uefa_2022 = spark.read.csv('/FileStore/Caso_2/uefa_2022.csv', header=True, inferSchema=True)

df_uefa_2020.show(2)
df_uefa_2021.show(2)
df_uefa_2022.show(2)

+--------------------+------------+-----+---------------+---------------+-----------------+--------------+---------------+---------------+---------------+---------------+----------------+----------------+--------------------+--------------------+--------------+--------------+------------------------+---------------+------------------------+-------------------+
|               STAGE|        DATE| PENS|PENS_HOME_SCORE|PENS_AWAY_SCORE|   TEAM_NAME_HOME|TEAM_NAME_AWAY|TEAM_HOME_SCORE|TEAM_AWAY_SCORE|POSSESSION_HOME|POSSESSION_AWAY|TOTAL_SHOTS_HOME|TOTAL_SHOTS_AWAY|SHOTS_ON_TARGET_HOME|SHOTS_ON_TARGET_AWAY|DUELS_WON_HOME|DUELS_WON_AWAY|PREDICTION_TEAM_HOME_WIN|PREDICTION_DRAW|PREDICTION_TEAM_AWAY_WIN|           LOCATION|
+--------------------+------------+-----+---------------+---------------+-----------------+--------------+---------------+---------------+---------------+---------------+----------------+----------------+--------------------+--------------------+--------------+-------------

1. Encuentra los 3 equipos que anotaron más goles jugando en su estadio en la UEFA Champions League 2020-21. El resultado debe contener dos columnas: TEAM_NAME_HOME y TEAM_HOME_SCORE ordenadas en orden descendente de TEAM_HOME_SCORE. Guarda la consulta (o el notebook) como TEAM_HOME_WITH_MOST_GOALS.

In [0]:
from pyspark.sql.functions import col, sum as spark_sum

df_union = df_uefa_2020.union(df_uefa_2021).union(df_uefa_2022)

# Filtro los datos para la temporada 2020-21
df_filtrado = df_union.filter(df_union['DATE'].like('%2020%'))

# Creo un df nuevo con los goles por equipo ordenado de manera descendente.
df_team_home_goals = df_filtrado.groupBy("TEAM_NAME_HOME") \
    .agg(spark_sum(col("TEAM_HOME_SCORE")).alias("TOTAL_HOME_GOALS")) \
    .orderBy(col("TOTAL_HOME_GOALS").desc())

# Creo un df nuevo con los 3 primeros y los muestro
top_3_equipos_mas_goleadores = df_team_home_goals.limit(3)
top_3_equipos_mas_goleadores.show()

+-------------------+----------------+
|     TEAM_NAME_HOME|TOTAL_HOME_GOALS|
+-------------------+----------------+
| Manchester United |              10|
|     Bayern Munich |               9|
|   Manchester City |               9|
+-------------------+----------------+



2. Encuentra el equipo con posesión mayoritaria la mayor cantidad de veces durante la UEFA Champions League 2021-22. El resultado debe incluir dos columnas: TEAM_NAME y GAME_COUNT, que es la cantidad de veces que el equipo tuvo posesión mayoritaria durante un partido de fútbol. Guarda esta consulta (o el notebook) como TEAM_WITH_MAJORITY_POSSESSION

In [0]:
from pyspark.sql.functions import when, count

df_filtrado = df_union.filter(df_union['DATE'].like('%2021%'))

# Creo un df con una columna en la que se indica por cada partido el nombre del equipo que más posesión obtuvo.
df_con_mayor_posesion = df_filtrado.withColumn(
    "TEAM_NAME",
    when(col("POSSESSION_HOME") > col("POSSESSION_AWAY"), col("TEAM_NAME_HOME"))
    .when(col("POSSESSION_AWAY") > col("POSSESSION_HOME"), col("TEAM_NAME_AWAY"))
)

# Creo un df nuevo con sólo las filas que no tengan valor nulo en la columna TEAM_NAME
df_limpio = df_con_mayor_posesion.filter(col("TEAM_NAME").isNotNull())

# A partir del df limpio de la línea anterior, creo otro con una columna "GAME_COUNT" que se obtiene de contar el número de veces que aparece el equipo en la columna "TEAM_NAME". Después lo ordeno de manera descendente.
df_con_numero_de_mayores_posesiones = df_limpio.groupBy("TEAM_NAME") \
    .agg(count("TEAM_NAME").alias("GAME_COUNT")) \
    .orderBy(col("GAME_COUNT").desc())

# Muestro sólo el primero que es el que más partidos con mayor posesión tuvo
df_con_numero_de_mayores_posesiones.show(1)

+-----------------+----------+
|        TEAM_NAME|GAME_COUNT|
+-----------------+----------+
| Manchester City |         6|
+-----------------+----------+
only showing top 1 row



3. Encuentra la lista de equipos de cada fase del juego que ganaron el duelo en un partido pero terminaron perdiendo el juego en el Campeonato de la UEFA 2022-23. El resultado debe contener dos columnas: STAGE y TEAM_LOST. Guarda la consulta (o el notebook) como TEAM_WON_DUEL_LOST_GAME_STAGE_WISE.

In [0]:
df_filtrado = df_union.filter(df_union['DATE'].like('%2022%'))

# Creo un df con una nueva columna "TEAM_LOST" en la que se almacenan los nombres de los equipos que han perdido el partido cuando su porcentaje de victorias es mayor al contrario.
df_equipos_con_mayor_porcentaje_pero_perdieron = df_filtrado.withColumn(
    "TEAM_LOST",
    when((col("DUELS_WON_HOME") > col("DUELS_WON_AWAY")) & (col("TEAM_HOME_SCORE") < col("TEAM_AWAY_SCORE")), col("TEAM_NAME_HOME"))
    .when((col("DUELS_WON_AWAY") > col("DUELS_WON_HOME")) & (col("TEAM_AWAY_SCORE") < col("TEAM_HOME_SCORE")), col("TEAM_NAME_AWAY"))
)

# Quito los posibles nulos y creo un df nuevo con ese filtro y sólo con las columnas "STAGE" y "TEAM_LOST"
df_final = df_equipos_con_mayor_porcentaje_pero_perdieron.filter(col("TEAM_LOST").isNotNull()).select("STAGE", "TEAM_LOST")
df_final.show(truncate=False)

+--------------------------+------------------+
|STAGE                     |TEAM_LOST         |
+--------------------------+------------------+
| Semi-finals second leg   |Man City          |
| Semi-finals first leg    |Villarreal        |
| Semi-finals first leg    |Real Madrid       |
| Quarter-finals first leg |Bayern            |
| Quarter-finals first leg |Atl�tico de Madrid|
| Round of 16 second leg   |Paris             |
| Round of 16 second leg   |Salzburg          |
| Round of 16 first leg    |Lille             |
| Round of 16 first leg    |Real Madrid       |
| Group stage: Matchday 1  |Chelsea           |
| Group stage: Matchday 1  |K�benhavn         |
| Group stage: Matchday 1  |Juventus          |
| Group stage: Matchday 1  |Maccabi Haifa     |
| Group stage: Matchday 1  |Rangers           |
| Group stage: Matchday 1  |Liverpool         |
| Group stage: Matchday 1  |Porto             |
| Group stage: Matchday 1  |Bayer Leverkusen  |
| Group stage: Matchday 1  |Viktoria Plz